In [1]:
import sympy as sp
import numpy as np

from SymPT import * 

from IPython.display import display, Math, Latex, Markdown

# Defining H

In [2]:
# ------------- Defining the symbols for the Hamiltonian ----------------
omega_x = sp.symbols('omega_x', positive=True, real=True) # Frequency of x confinement
lx = sp.Symbol('l_x', positive=True, real=True)         # Length of x confinement
m = sp.Symbol('m', positive=True, real=True)                    # Effective mass of the particle on Si
e = -sp.Symbol('e', positive=True, real=True)                    # Absolute value of the electron  charge

ky = RDSymbol('k_y', positive=True, real=True)  
kB = RDSymbol('k_B', positive=True, real=True)  
B = sp.symbols('B', real=True)          # Total Magnetic field perpendicular to the plane of xy confinement B = B0 + b0, where b0 is genereted by the skyrmion
omega_y = sp.symbols('omega_y', positive=True, real=True)               # Frequency of y confinement
omega_c = sp.symbols('omega_c', positive=True, real=True)                # Frequency of omega_c = |e|B/m


# Effective frequencies affter diagonalization of H0 using the minimal coupling substitution
# Omega_3_2 = sp.sqrt((omega_x**2 - omega_y**2)**2 + 2 * omega_c**2 * (omega_x**2 + omega_y**2) + omega_c**4)
omega_p = sp.symbols('omega_+', positive=True, real=True)            #1/sp.sqrt(2) * sp.sqrt(omega_x**2 + omega_y**2 + omega_c**2 + Omega_3_2)
omega_m = sp.symbols('omega_-', positive=True, real=True)             #1/sp.sqrt(2) * sp.sqrt(omega_x**2 + omega_y**2 + omega_c**2 - Omega_3_2)

In [3]:
Omega_3 = sp.symbols('Omega_3', positive=True, real=True)

Omega_x = sp.sqrt(omega_x**2 + sp.Rational(1, 4) * omega_c**2)
Omega_y = sp.sqrt(omega_y**2 + sp.Rational(1, 4) * omega_c**2)

subs_simplifications = {
    Omega_3**2 : omega_p**2 - omega_m**2,
    omega_x**2 + omega_y**2 + omega_c**2 : omega_p**2 + omega_m**2,
    omega_x**2 + 3*omega_y**2 + omega_c**2 : omega_p**2 + omega_m**2 + 2*omega_y**2,
    3*omega_x**2 + omega_y**2 + omega_c**2 : omega_p**2 + omega_m**2 + 2*omega_x**2,
    omega_x * omega_y : omega_p * omega_m,
}

c1 = sp.sqrt(sp.Rational(1,2) * (Omega_x**2 + 3*Omega_y**2 + Omega_3**2)/(Omega_x**2 + Omega_y**2)).simplify().factor().subs(subs_simplifications)
c2 = sp.sqrt(sp.Rational(1,2) * (3*Omega_x**2 + Omega_y**2 - Omega_3**2)/(Omega_x**2 + Omega_y**2)).simplify().factor().subs(subs_simplifications)
Omega_1 = (sp.Rational(1,2) * sp.sqrt(3*Omega_x**2 + Omega_y**2 + Omega_3**2)).factor().subs(subs_simplifications)
Omega_2 = (sp.Rational(1,2) * sp.sqrt(Omega_x**2 + 3*Omega_y**2 - Omega_3**2)).factor().subs(subs_simplifications)

display((c1**2/Omega_1**2).factor())
display((c2**2/Omega_2**2).factor())

4*(omega_+**2 + omega_y**2)/((omega_+**2 + omega_x**2)*(omega_c**2 + 2*omega_x**2 + 2*omega_y**2))

4*(omega_-**2 + omega_x**2)/((omega_-**2 + omega_y**2)*(omega_c**2 + 2*omega_x**2 + 2*omega_y**2))

In [4]:
# ------------- Defining the position momentum basis for the Hamiltonian ----------------
# Defining the bosonic operators of the new basis after diagonalization of H0
# a_+ and a_- are the bosonic operators of the new basis

a_1 = BosonOp('a_+')
ad_1 = Dagger(a_1)
a_2 = BosonOp('a_-')
ad_2 = Dagger(a_2)

# -----------------------------------------------------------------------------
# Defining the position operators x and y in the new basis after diagonalization of H0 and using the bosonic operators of the new basis
theta = sp.Symbol('theta', real=True)               # Parameter used on the diagonalization of H0
beta = (- m * sp.sqrt(Omega_x**2 + Omega_y**2) / sp.sqrt(2)).factor()
alpha = 1 / beta


c1_Omega1 = sp.sqrt((c1**2/Omega_1**2).factor())
c2_Omega2 = sp.sqrt((c2**2/Omega_2**2).factor())

# Substitutions for the lenghts of the confinement
subs_lenght = {
    sp.sqrt(hbar / (m * omega_x)): lx,
    sp.sqrt( m * omega_x / hbar): 1 / lx,
    sp.sqrt( m * omega_x * hbar): hbar / lx,
}

# Position Operators
x = Operator('x')
y = Operator('y')
px = Operator('p_x')
py = Operator('p_y')

# Position operators in the new basis
x_T  = (sp.sqrt(hbar/(2*m) * c1_Omega1) * sp.cos(theta) * (ad_1 + a_1) - sp.I * sp.sqrt(hbar*m * 1 / (2 * c2_Omega2)) * alpha * sp.sin(theta) * (ad_2 - a_2))
y_T  = (sp.sqrt(hbar/(2*m) * c2_Omega2) * sp.cos(theta) * (ad_2 + a_2) - sp.I * sp.sqrt(hbar*m * 1 / (2 * c1_Omega1)) * alpha * sp.sin(theta) * (ad_1 - a_1))
px_T = (sp.sqrt(hbar/(2*m)* c2_Omega2) * beta * sp.sin(theta) * (ad_2 + a_2) + sp.I * sp.sqrt(hbar * m * 1 / (2 * c1_Omega1)) * sp.cos(theta) * (ad_1 - a_1))
py_T = (sp.sqrt(hbar/(2*m)* c1_Omega1) * beta * sp.sin(theta) * (ad_1 + a_1) + sp.I * sp.sqrt(hbar * m * 1 / (2 * c2_Omega2)) * sp.cos(theta) * (ad_2 - a_2))

transformation = {
    x : x_T,
    y : y_T,
    px : px_T,
    py : py_T
}

a, b = sp.symbols('a b', real=True, positive=True)
subs_theta = {
    theta : sp.Rational(1,2) * sp.atan(-a/b)
}

subs_theta.update({
    sp.cos(theta) * sp.sin(theta) : (sp.cos(theta) * sp.sin(theta)).subs(subs_theta).trigsimp().simplify(),
    sp.sin(theta)**2 : (sp.sin(theta)**2).subs(subs_theta).trigsimp().simplify(),
    sp.cos(theta)**2 : (sp.cos(theta)**2).subs(subs_theta).trigsimp().simplify(),
})


display_dict(transformation)

# -----------------------------------------------------------------------------
# ------------- Defining the spin basis for the Hamiltonian ----------------

Spin = RDBasis('sigma', 2)
s0, sx, sy, sz = Spin.basis # Pauli operators

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Lab Frame

In [5]:
Omega_z = sp.Symbol('Omega_z', positive=True, real=True)          # Zeeman energy   Omega_z = e / m0 * (B + b0)

n1 = RDSymbol('\\tilde{n}_1', order=1, positive=True, integer=True)          # b1=n1/lx -> \tilde{n}_1 = e/m0 * b1
n2 = RDSymbol('\\tilde{n}_2', order=2, positive=True, integer=True)          # b2=n2/lx**2 -> \tilde{n}_2 = e/m0 * b2

H0 = ((sp.factor_terms((hbar * omega_p * ad_1 * a_1 + hbar * omega_m * ad_2 * a_2).expand()).expand() - sp.Rational(1,2) * hbar * Omega_z * sz)).expand()
V1 = (sp.Rational(1, 2) * hbar * n1 * (x * sx + y * sy)).subs(transformation).expand()
V2 = (sp.Rational(1, 2) * hbar * n2 * (x**2 + y**2) * sz).subs(transformation).expand()

In [ ]:
kE = RDSymbol('k_E', order=-2/3, positive=True, real=True)
omega_D = sp.Symbol('omega_D', positive=True, real=True)


E0 = - hbar * omega_x / (2 * lx) * 1/ kE**(3/2) * sp.cos(omega_D * t)
H_drive = E0 * x_T   # Drive Hamiltonian in units of hbar*omega_x


H = H0 + V1 + V2 + H_drive

In [25]:
E0

-hbar*omega_x*cos(t*omega_D)/(2*k_E**1.5*l_x)

### Full Diagonalization and then Rotate

In [21]:
Eff_frame = EffectiveFrame(H0, V1 + V2, subspaces=[Spin])
Eff_frame.solve(max_order=2, method='FD')
result = Eff_frame.get_H('matrix')

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│  a_+   │ Bosonic │      ∞      │
├────────┼─────────┼─────────────┤
│  a_-   │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 


The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  2.21it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to matrix form: 100%|██████████| 3/3 [00:00<00:00, 761.54it/s]


In [22]:
H_eff_drive = Eff_frame.rotate(H_drive)

subs_ab = {
    sp.sqrt(a**2 + b**2) : omega_p**2 - omega_m**2,
}

subs_a = {
    a: omega_c*sp.sqrt(2*Omega_x**2 + 2*Omega_y**2),
}

subs_b = {
    b: omega_x**2 - omega_y**2,
}

H_eff_drive = group_by_operators(H_eff_drive)
Hx = H_eff_drive.get(sx).collect([sp.sin(theta)*sp.cos(theta), sp.sin(theta)**2, sp.cos(theta)**2]).subs(subs_theta).subs(subs_ab).subs(subs_a).subs(subs_b)
Hx = sp.Add(*[sp.factor_terms(sp.expand_complex(sp.factor_terms(hx)).factor()) for hx in Hx.as_ordered_terms()])

Projecting to operator form: 100%|██████████| 5/5 [00:00<00:00, 27.34it/s]


In [23]:
Hx

\tilde{n}_1*hbar**2*Omega_z*omega_c*omega_x*cos(t*omega_D)/(4*k_E**1.5*l_x*m*(Omega_z - omega_+)*(Omega_z + omega_+)*(Omega_z - omega_-)*(Omega_z + omega_-)) - \tilde{n}_1*hbar**2*omega_+*omega_x*sqrt(omega_+**2 + omega_y**2)*(omega_+**2 - omega_-**2 + omega_x**2 - omega_y**2)*cos(t*omega_D)/(4*k_E**1.5*l_x*m*(Omega_z - omega_+)*(Omega_z + omega_+)*(omega_+ - omega_-)*(omega_+ + omega_-)*sqrt(omega_+**2 + omega_x**2)*sqrt(omega_c**2 + 2*omega_x**2 + 2*omega_y**2)) - \tilde{n}_1*hbar**2*omega_-*omega_x*sqrt(omega_-**2 + omega_y**2)*(omega_+**2 - omega_-**2 - omega_x**2 + omega_y**2)*cos(t*omega_D)/(4*k_E**1.5*l_x*m*(Omega_z - omega_-)*(Omega_z + omega_-)*(omega_+ - omega_-)*(omega_+ + omega_-)*sqrt(omega_-**2 + omega_x**2)*sqrt(omega_c**2 + 2*omega_x**2 + 2*omega_y**2))

In [24]:
Hx.subs({omega_c : 0, omega_p : omega_x, omega_m : omega_y}).cancel().factor()

-\tilde{n}_1*hbar**2*omega_x*cos(t*omega_D)/(4*k_E**1.5*l_x*m*(Omega_z - omega_x)*(Omega_z + omega_x))

### Block Diagonalization Time Dependent

In [7]:
Eff_frame = EffectiveFrame(H0 + V2, V1 + H_drive, subspaces=[Spin])
Eff_frame.solve(max_order=2, method='SW')
result = Eff_frame.get_H('matrix')

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│  a_+   │ Bosonic │      ∞      │
├────────┼─────────┼─────────────┤
│  a_-   │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  2.76it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to matrix form: 100%|██████████| 11/11 [00:00<00:00, 532.47it/s]


In [8]:
result = result.expand()

In [9]:
H_eff = result[0,0] * sp.Rational(1,2) * (1 + sz) + result[1,1] * sp.Rational(1,2) * (1 - sz) + result[0,1] * sp.Rational(1,2) * (sx + sp.I * sy) + result[1,0] * sp.Rational(1,2) * (sx - sp.I * sy)

In [10]:
H_eff = group_by_operators(H_eff)

In [ ]:
subs_ab = {
    sp.sqrt(a**2 + b**2) : omega_p**2 - omega_m**2,
}

subs_a = {
    a: omega_c*sp.sqrt(2*Omega_x**2 + 2*Omega_y**2),
}

subs_b = {
    b: omega_x**2 - omega_y**2,
}

Hx = H_eff.get(sx).collect([sp.sin(theta)*sp.cos(theta), sp.sin(theta)**2, sp.cos(theta)**2]).subs(subs_theta).subs(subs_ab).subs(subs_a).subs(subs_b)
Hx = sp.Add(*[sp.factor_terms(sp.expand_complex(sp.factor_terms(hx)).factor()) for hx in Hx.as_ordered_terms()])

In [12]:
sp.factor_terms(sp.Add(*Hx.as_ordered_terms()[1:]))

\tilde{n}_1*hbar*(omega_+*sqrt(omega_+**2 + omega_y**2)*(Omega_z**2 - 2*omega_+**2 + omega_D**2)*(omega_+**2 - omega_-**2 + omega_x**2 - omega_y**2)/((Omega_z - omega_+)*(Omega_z + omega_+)*(omega_+ - omega_D)*(omega_+ + omega_D)*sqrt(omega_+**2 + omega_x**2)) + omega_-*sqrt(omega_-**2 + omega_y**2)*(Omega_z**2 - 2*omega_-**2 + omega_D**2)*(omega_+**2 - omega_-**2 - omega_x**2 + omega_y**2)/((Omega_z - omega_-)*(Omega_z + omega_-)*(omega_- - omega_D)*(omega_- + omega_D)*sqrt(omega_-**2 + omega_x**2)))*cos(t*omega_D)/(8*k_E**1.5*l_x*m*(omega_+ - omega_-)*(omega_+ + omega_-)*sqrt(omega_c**2 + 2*omega_x**2 + 2*omega_y**2))